# CIFAR-10 Classification with Different Deep Learning Architectures

In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt
import numpy as np

# Set random seed for reproducibility
torch.manual_seed(42)

## 2. Load CIFAR-10 Dataset

In [2]:
# Transform and load dataset
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=4,
                                         shuffle=False, num_workers=2)

classes = trainset.classes

## 3. Define Models

In [3]:
# Model 1: Logistic Regression
class LogisticRegression(nn.Module):
    def __init__(self):
        super(LogisticRegression, self).__init__()
        self.fc = nn.Linear(32 * 32 * 3, 10)

    def forward(self, x):
        x = x.view(-1, 32 * 32 * 3)
        return self.fc(x)

# Model 2: One Hidden Layer Fully Connected
class OneHiddenLayerNN(nn.Module):
    def __init__(self, hidden_size=512):
        super(OneHiddenLayerNN, self).__init__()
        self.fc1 = nn.Linear(32 * 32 * 3, hidden_size)
        self.fc2 = nn.Linear(hidden_size, 10)

    def forward(self, x):
        x = x.view(-1, 32 * 32 * 3)
        x = F.relu(self.fc1(x))
        return self.fc2(x)

# Model 3: CNN with one Conv Layer + MaxPool
class ConvNet(nn.Module):
    def __init__(self, out_channels=100, kernel_size=5, pool_size=14):
        super(ConvNet, self).__init__()
        self.conv1 = nn.Conv2d(3, out_channels, kernel_size)
        self.pool = nn.MaxPool2d(pool_size)
        self.fc = nn.Linear(out_channels * 3 * 3, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = x.view(-1, x.numel() // x.size(0))
        return self.fc(x)

## 4. Training Function

In [4]:
def train_model(model, optimizer, criterion, trainloader, testloader, epochs=10):
    train_acc, test_acc = [], []
    for epoch in range(epochs):
        correct, total = 0, 0
        model.train()
        for inputs, labels in trainloader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
        acc = 100 * correct / total
        train_acc.append(acc)
        print(f'Epoch {epoch+1}, Train Accuracy: {acc:.2f}%')

        # Evaluate on test set
        correct, total = 0, 0
        model.eval()
        with torch.no_grad():
            for inputs, labels in testloader:
                outputs = model(inputs)
                _, predicted = torch.max(outputs, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
        acc = 100 * correct / total
        test_acc.append(acc)
        print(f'Test Accuracy: {acc:.2f}%')
    return train_acc, test_acc

## 5. Train and Compare Models

In [ ]:
# Example: Train Logistic Regression
model = LogisticRegression()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
criterion = nn.CrossEntropyLoss()
train_acc, test_acc = train_model(
    model, optimizer, criterion, trainloader, testloader, epochs=10)

Epoch 1, Train Accuracy: 32.03%
Test Accuracy: 35.37%
Epoch 2, Train Accuracy: 34.33%
Test Accuracy: 34.85%


## 6. Plot Results

In [ ]:
def plot_accuracy(train_acc, test_acc, title='Accuracy vs Epoch'):
    plt.figure(figsize=(8,5))
    plt.plot(train_acc, label='Train Accuracy')
    plt.plot(test_acc, label='Test Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy (%)')
    plt.title(title)
    plt.legend()
    plt.grid(True)
    plt.show()

## 7. Train Train and Plot All Models

In [ ]:
# Train FCN with 1 hidden layer
model2 = OneHiddenLayerNN(hidden_size=512)
optimizer2 = optim.SGD(model2.parameters(), lr=0.001, momentum=0.9)
train_acc2, test_acc2 = train_model(model2, optimizer2, criterion, trainloader, testloader)
plot_accuracy(train_acc2, test_acc2, title='One Hidden Layer FCN')

# Train CNN model
model3 = ConvNet(out_channels=100, kernel_size=5, pool_size=14)
optimizer3 = optim.SGD(model3.parameters(), lr=0.001, momentum=0.9)
train_acc3, test_acc3 = train_model(model3, optimizer3, criterion, trainloader, testloader)
plot_accuracy(train_acc3, test_acc3, title='CNN with MaxPool')

## Extra Credit 

In [ ]:
class TunedCNN(nn.Module):
    def __init__(self):
        super(TunedCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 64, 3, padding=1)
        self.conv2 = nn.Conv2d(64, 128, 3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.dropout = nn.Dropout(0.25)
        self.fc1 = nn.Linear(128 * 8 * 8, 512)
        self.fc2 = nn.Linear(512, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 128 * 8 * 8)
        x = self.dropout(x)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

       

In [ ]:
model4 = TunedCNN()
optimizer4 = optim.Adam(model4.parameters(), lr=0.001)
train_acc4, test_acc4 = train_model(model4, optimizer4, criterion, trainloader, testloader, epochs=20)
plot_accuracy(train_acc4, test_acc4, title='Tuned CNN (Extra Credit)')
